<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%ED%86%B5%ED%95%A9/(%ED%86%B5%ED%95%A9)_%EC%9A%94%EC%95%BD%EB%AC%B8_%EC%A6%9D%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [2]:
pip install transformers sentencepiece torch rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e7cba7a602d540701ebe42bd375c819a6e70bd2e951fb0e0a891d43fdd9e892a
  

In [3]:
!pip install konlpy
!pip install textrankr
!pip install JPype1>=0.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 21.4 MB/s eta 0:00:00


# 요약문 알고리즘

## ① TextRank

In [12]:
import pandas as pd
import re
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

def textrank_summary(text, ratio=0.2, num_summaries=10):
    if not isinstance(text, str) or text.strip() == "":
        return [""] * num_summaries  # 빈 문자열 리스트 반환
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    sentences = [s.strip() for s in sentences if s.strip()]
    if len(sentences) == 0:
        return [""] * num_summaries  # 유효한 문장이 없을 경우
    top_n = max(1, int(len(sentences) * ratio))
    vectorizer = TfidfVectorizer()
    try:
        sentence_vectors = vectorizer.fit_transform(sentences)
    except ValueError:
        return [text] * num_summaries  # 빈 vocabulary 오류 발생 시 원문 반환
    similarity_matrix = cosine_similarity(sentence_vectors, sentence_vectors)
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summaries = []
    for _ in range(num_summaries):
        # 문장 샘플링 방식 변경 (다양성 확보)
        sampled_sentences = random.sample(ranked_sentences[:min(len(ranked_sentences), 10)], top_n)
        sampled_sentences = sorted(sampled_sentences, key=lambda x: sentences.index(x[1]))  # 원래 문장 순서 유지
        summaries.append(' '.join([s[1] for s in sampled_sentences]))
    return summaries

## ② KoBART

In [13]:
!git clone https://github.com/seujung/KoBART-summarization.git

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 155 (delta 49), reused 44 (delta 40), pack-reused 83 (from 1)
Receiving objects: 100% (155/155), 37.24 MiB | 42.32 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [14]:
!pip install loguru
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 39.0 MB/s eta 0:00:00


In [16]:
cd /content/drive/MyDrive/KoBART-summarization

[Errno 2] No such file or directory: '/content/drive/MyDrive/KoBART-summarization'
/content


In [17]:
!pip install -r requirements.txt
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# 클론한 파일 제공 데이터
%cd /content/KoBART-summarization/data
!tar -zxvf train.tar.gz
!tar -zxvf test.tar.gz
!ls
!pip install git+https://github.com/SKT-AI/KoBART

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu101 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1

In [18]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [28]:
# KoBART 기반 추상적 요약
def Kobart_summary(text, num_summaries=10, max_length=200, num_beams=50, temperature=1.0, top_k=50, top_p=0.95):
    if not isinstance(text, str) or text.strip() == "":
        return [""] * num_summaries  # 빈 문자열 반환

    input_ids = tokenizer.encode(text, return_tensors="pt")

    # KoBART 요약 생성
    output = model.generate(
        input_ids,
        eos_token_id=1,
        max_length=max_length,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        num_return_sequences=num_summaries,  # 여러 개의 요약 생성
        temperature=temperature,  # 다양성 조절
        top_k=top_k,
        top_p=top_p
    )
    summaries = [tokenizer.decode(out, skip_special_tokens=True) for out in output]
    return summaries

# 2% 불균형데이터의 요약문 증강 데이터

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   NSMC_0.02_증분대상100개.csv   NSMC_0.5_증분대상_2500개.csv	 NSMC_train.csv
'My Drive'	    NSMC_0.1_증분대상_500개.csv   NSMC_0.8_증분대상_1000개.csv	 ★졸업논문


In [27]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/NSMC_0.02_증분대상100개.csv', encoding='utf-8-sig')

In [29]:
#ratio 0.3, num_summaries=10 / ratio 0.4, num_summaries=10 / ratio 0.5, num_summaries=5
ratio=0.3
num_summaries=20
summary_cols=[f'ext0.3_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(textrank_summary(x, ratio=ratio, num_summaries=num_summaries)))

ratio=0.4
num_summaries=20
summary_cols=[f'ext0.4_{i+1}' for i in range(num_summaries)]

ratio=0.5
num_summaries=10
summary_cols=[f'ext0.5_{i+1}' for i in range(num_summaries)]

In [30]:
num_summaries=49
summary_cols = [f'abs_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(Kobart_summary(x, num_summaries=num_summaries)))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [31]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'document', 'label', 'ext0.3_1',
       'ext0.3_2', 'ext0.3_3', 'ext0.3_4', 'ext0.3_5', 'ext0.3_6', 'ext0.3_7',
       'ext0.3_8', 'ext0.3_9', 'ext0.3_10', 'ext0.3_11', 'ext0.3_12',
       'ext0.3_13', 'ext0.3_14', 'ext0.3_15', 'ext0.3_16', 'ext0.3_17',
       'ext0.3_18', 'ext0.3_19', 'ext0.3_20', 'abs_1', 'abs_2', 'abs_3',
       'abs_4', 'abs_5', 'abs_6', 'abs_7', 'abs_8', 'abs_9', 'abs_10',
       'abs_11', 'abs_12', 'abs_13', 'abs_14', 'abs_15', 'abs_16', 'abs_17',
       'abs_18', 'abs_19', 'abs_20', 'abs_21', 'abs_22', 'abs_23', 'abs_24',
       'abs_25', 'abs_26', 'abs_27', 'abs_28', 'abs_29', 'abs_30', 'abs_31',
       'abs_32', 'abs_33', 'abs_34', 'abs_35', 'abs_36', 'abs_37', 'abs_38',
       'abs_39', 'abs_40', 'abs_41', 'abs_42', 'abs_43', 'abs_44', 'abs_45',
       'abs_46', 'abs_47', 'abs_48', 'abs_49'],
      dtype='object')

In [32]:
df.to_csv('/content/gdrive/MyDrive/요약문 증분 완료_0.02.csv', encoding='utf-8-sig')

# 10% 불균형 데이터

In [33]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/NSMC_0.1_증분대상_500개.csv', encoding='utf-8-sig')

In [34]:
#ratio 0.3, num_summaries=10 / ratio 0.4, num_summaries=10 / ratio 0.5, num_summaries=5
ratio=0.3
num_summaries=5
summary_cols=[f'ext0.3_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(textrank_summary(x, ratio=ratio, num_summaries=num_summaries)))

ratio=0.4
num_summaries=5
summary_cols=[f'ext0.4_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(textrank_summary(x, ratio=ratio, num_summaries=num_summaries)))

In [ ]:
num_summaries=9
summary_cols = [f'abs_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(Kobart_summary(x, num_summaries=num_summaries)))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
df.to_csv('/content/gdrive/MyDrive/요약문 증분 완료_0.1.csv', encoding='utf-8-sig')

# 50% 데이터 증분하기

In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/NSMC_0.5_증분대상_2500개.csv', encoding='utf-8-sig')

In [ ]:
#ratio 0.3, num_summaries=10 / ratio 0.4, num_summaries=10 / ratio 0.5, num_summaries=5
ratio=0.3
num_summaries=1
summary_cols=[f'ext0.3_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(textrank_summary(x, ratio=ratio, num_summaries=num_summaries)))

In [ ]:
num_summaries=1
summary_cols = [f'abs_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(Kobart_summary(x, num_summaries=num_summaries)))

In [ ]:
df.to_csv('/content/gdrive/MyDrive/요약문 증분 완료_0.5.csv', encoding='utf-8-sig')

# 80% 데이터 증분하기

In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/NSMC_0.8_증분대상_1000개.csv', encoding='utf-8-sig')

In [ ]:
#ratio 0.3, num_summaries=10 / ratio 0.4, num_summaries=10 / ratio 0.5, num_summaries=5
ratio=0.3
num_summaries=1
summary_cols=[f'ext0.3_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(textrank_summary(x, ratio=ratio, num_summaries=num_summaries)))

In [ ]:
num_summaries=1
summary_cols = [f'abs_{i+1}' for i in range(num_summaries)]
df[summary_cols] = df['document'].apply(lambda x: pd.Series(Kobart_summary(x, num_summaries=num_summaries)))

In [ ]:
df.to_csv('/content/gdrive/MyDrive/요약문 증분 완료_0.8.csv', encoding='utf-8-sig')

# 요약문 평가
- ROUGE (Recall-Oriented Understudy for Gisting Evaluation) 점수는 자동 요약 및 자연어 생성 모델의 성능을 평가하는 대표적인 방법 중 하나입니다. 주요 ROUGE 지표는 다음과 같습니다.
- ROUGE-1: 요약된 문장에서 정답과 일치하는 단어 단위(n=1)의 개수를 기반으로 측정
- ROUGE-2: 정답과 요약이 공유하는 2-그램(n=2) 단위의 개수 기반 평가
- ROUGE-L: 정답과 요약된 문장 간 가장 긴 공통 부분 문자열(LCS, Longest Common Subsequence) 기반 평가

In [ ]:
df=df_2

In [ ]:
df.columns

Index(['demographic_category', 'demographic_group', 'context', 'context_en',
       'sentence', 'sentence_en', 'context_label', 'context_sub_label',
       'sentence_label', 'sentence_sub_label', 'annotated_demo', 'text',
       'summary_ext', 'summary_abs'],
      dtype='object')

In [ ]:
from rouge_score import rouge_scorer
import pandas as pd

# ROUGE 점수 계산 함수
def calculate_rouge(df, summary_col, reference_col):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, pred in zip(df[reference_col], df[summary_col]):
        score = scorer.score(ref, pred)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    return {metric: sum(values)/len(values) for metric, values in scores.items()}

# 평가 수행
rouge_scores_abs = calculate_rouge(df, 'summary_abs', 'text') #정답데이터 : summary (=text)
rouge_scores_ext = calculate_rouge(df, 'summary_ext', 'text') #정답데이터 : summary (=text)

# 결과 출력
print("ROUGE 점수 (KoBART 요약)", rouge_scores_abs)
print("ROUGE 점수 (TextRank 요약)", rouge_scores_ext)


ROUGE 점수 (KoBART 요약) {'rouge1': 0.15833333333333333, 'rouge2': 0.0, 'rougeL': 0.15833333333333333}
ROUGE 점수 (TextRank 요약) {'rouge1': 0.5, 'rouge2': 0.0, 'rougeL': 0.5}
